In [3]:
import findspark

In [11]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7/')

In [12]:
import pyspark

In [13]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder.appName('my_log_reg').getOrCreate()

In [17]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [18]:
from pyspark.ml.classification import LogisticRegression

In [19]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [25]:
my_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                100|
|   mean|               0.57|
| stddev|0.49756985195624287|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [22]:
## Lets set our model from SparkML
my_log_reg_model = LogisticRegression()

In [23]:
## We dont bother now about the splitting of Train and test data
## Now just training
fitted_logreg = my_log_reg_model.fit(my_data)

In [26]:
## Results
log_summary = fitted_logreg.summary

In [31]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [32]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111048...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126969...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170264...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200596...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695572...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102604...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215612...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

Lets introduce the concept of Evaluators

In [34]:
## lets split the data
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

In [36]:
final_model = LogisticRegression()

In [37]:
fit_final = final_model.fit(lr_train)

In [38]:
##Now evaluation
predictions_and_labels = fit_final.evaluate(lr_test)

In [40]:
predictions_and_labels.predictions.show()
##only diff from log_summary is we evaluated on test data

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,124...|[14.8696271738648...|[0.99999965149997...|       0.0|
|  0.0|(692,[127,128,129...|[21.2279329929299...|[0.99999999939629...|       0.0|
|  0.0|(692,[128,129,130...|[22.5259565957511...|[0.99999999983514...|       0.0|
|  0.0|(692,[152,153,154...|[17.3504825676192...|[0.99999997084042...|       0.0|
|  0.0|(692,[152,153,154...|[31.3375560506598...|[0.99999999999997...|       0.0|
|  0.0|(692,[152,153,154...|[13.4208490319196...|[0.99999851612035...|       0.0|
|  0.0|(692,[153,154,155...|[28.6079911687143...|[0.99999999999962...|       0.0|
|  1.0|(692,[99,100,101,...|[-8.2045958429610...|[2.73319477482253...|       1.0|
|  1.0|(692,[123,124,125...|[-22.163209923842...|[2.36941006362746...|       1.0|
|  1.0|(692,[123

In [41]:
## lets explore the evaluation here
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [42]:
my_eval = BinaryClassificationEvaluator()

In [45]:
my_final_roc = my_eval.evaluate(predictions_and_labels.predictions)

In [46]:
## Area under the curve
my_final_roc

1.0